In [1]:
from Haldane_spin_half import *

In [2]:
L = 5 # should be odd
bond = 5
HaldaneSpinHalf = HaldaneSpinHalf(L = L, bond = bond)

In [ ]:
state, targets = HaldaneSpinHalf.generate_train_set()

In [7]:
state.shape

(220,)

In [15]:
train_points = pd.read_csv('~/4th_project_with_UCL/supervised learning/Second_hamiltonian/dataset/regular_train_set.csv')

In [ ]:
import gc
import h5py
import quimb as qu
import quimb.tensor as qtn
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import *
import matplotlib.pyplot as plt
from Haldane_spin_half import *

/home/mehran/.local/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [8]:
L = 11  # number of particle which has to be odd for calculate faster
ls_train = 11 # the scale of dividing the range of h and k for train part (including projected states)
ls_test = 11 # the scale of dividing the range of h and k for test part (including DMRG states)

In [9]:
train_set = ANNNI(L = L, ls = ls_train).generate_train_set() #ts[0] = DMRG_state, ts[1] = DMRG_target, ts[2] = project_state,ts[3] = projection_target, ts[4] = h1_h2
gc.collect(2)

495

In [10]:
with h5py.File("projection_target.hdf5", "w") as f:
    f.create_dataset("y", data = train_set[3])

In [11]:
test_set = ANNNI(L = L, ls = ls_test).generate_test_set()#ts[0]=DMRG_state, ts[1]=h1h2

In [12]:
def gram_train():
    Xt = train_set[2]
    d = len(Xt)
    gram_matrix_train = np.zeros((d,d))
    for i in range(d):
        for j in range(d):
            gram_matrix_train[i,j] = np.abs(Xt[i].H @ Xt[j])**2/ (np.abs(Xt[i].H @ Xt[i])*np.abs(Xt[j].H @ Xt[j]))
    return gram_matrix_train

gtrain = gram_train()

with h5py.File("gram_train.hdf5", "w") as f:
    f.create_dataset("gram_train", data = gram_train())

gc.collect(2)

0

In [13]:
def gram_test():
    Xte = test_set[0]
    d1 = len(Xte)
    Xtr = train_set[2]
    d2 = len(Xtr)
    gram_matrix_test = np.zeros((d1,d2))
    for i in range(d1):
        for j in range(d2):
            gram_matrix_test[i,j] = np.abs((Xte[i].H @ Xtr[j]))**2/ (np.abs((Xte[i].H @ Xte[i]))*np.abs((Xtr[j].H @ Xtr[j])))
    return gram_matrix_test

gtest = gram_test()

with h5py.File("gram_test.hdf5", "w") as f:
    f.create_dataset("gram_test", data = gram_test())

gc.collect(2)

0